<a href="https://colab.research.google.com/github/youngeun1207/keras/blob/main/SHL/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def loadData(usr, day):
  dir_usr = ["user1", "user2", "user3"]
  dir_day = ["day1", "day2", "day3"]
  file_name = ["Hand_Motion.txt","Hand_GPS.txt","Hand_Location.txt", "Label.txt"]
  path = "/content/drive/MyDrive/SHLdatasets"

  motion_path = path + "/" + dir_usr[usr] + "/" + dir_day[day] + "/" + file_name[0]
  gps_path = path + "/" + dir_usr[usr] + "/" + dir_day[day] + "/" + file_name[1]
  location_path = path + "/" + dir_usr[usr] + "/" + dir_day[day] + "/" + file_name[2]
  label_path = path + "/" + dir_usr[usr] + "/" + dir_day[day] + "/" + file_name[3]

  print(motion_path)

  # 'Time', 'Acc_X', 'Acc_Y', 'Acc_Z', 'Gyr_X', 'Gyr_Y', 'Gyr_Z', 'Mag_X', 'Mag_Y', 'Mag_Z'
  full_motion = pd.read_csv(motion_path, sep=" ",header=None, usecols=list(range(10)))
  # 'Time', 'GPS_SNR' , 'GPS_Az, ', 'GPS_Elev', 
  full_gps = pd.read_csv(gps_path, sep=" ",header=None, usecols=[0,4,5,6])
  # 'Time, Accuracy, Latitude, Longitude, Altitude'
  full_location = pd.read_csv(location_path, sep=" ",header=None, usecols=[0,3,4,5,6])
  # Null=0, Still=1, Walking=2, Run=3, Bike=4, Car=5, Bus=6, Train=7, Subway=8
  full_label = pd.read_csv(label_path, sep=" ",header=None, usecols=[1])

  full_motion.columns = ['Time', 'Acc_X', 'Acc_Y', 'Acc_Z', 'Gyr_X', 'Gyr_Y', 'Gyr_Z', 'Mag_X', 'Mag_Y', 'Mag_Z']
  full_gps.columns = ['Time', 'GPS_SNR' , 'GPS_Az, ', 'GPS_Elev']
  full_location.columns = ['Time', 'Accuracy', 'Latitude', 'Longitude', 'Altitude']
  full_label.columns = ['Label']

  return full_motion, full_gps, full_location, full_label

In [ ]:
import pandas as pd
import numpy as np
from keras.utils.np_utils import to_categorical

def mergeData(full_motion, full_gps, full_location):
  full_motion.iloc[:, 0] = full_motion.iloc[:, 0].astype(int)
  full_gps.iloc[:, 0] = full_gps.iloc[:, 0] // 10 * 10
  full_location.iloc[:, 0] = full_location.iloc[:, 0] // 10 * 10

  full_data = pd.merge(full_motion, full_gps, on='Time', how='left')
  full_data = pd.merge(full_data, full_location, on='Time', how='left')
  
  full_data.fillna(method='ffill', limit=5, axis=0, inplace=True)
  full_data.fillna(method='bfill', limit=5, axis=0, inplace=True)
  full_data.drop_duplicates('Time', keep='first', inplace=True)

  return full_data

In [ ]:
def toNumpy(data, label):
  final_data = data.to_numpy()
  final_data = final_data.reshape(-1,1,17)

  label_np = label['Label'].T
  label_np = label_np.to_numpy()

  # one-hot encoding
  final_label = np.zeros(shape=(len(label_np), 9))
  
  label_one_hot = to_categorical(label_np)
  # label_one_hot = label_one_hot[0]
  for i in range(len(label_one_hot)):
    final_label[i] = np.pad(label_one_hot[i], (0,9 - len(label_one_hot[i])), constant_values = 0)

  return final_data, final_label

In [ ]:
def getData(usr, day):
  usr-=1
  day-=1
  full_motion, full_gps, full_location, label = loadData(usr, day)
  data = mergeData(full_motion, full_gps, full_location)
  np_data, np_label = toNumpy(data, label)
  return np_data, np_label

In [ ]:
# user 1,2 데이터로 훈련
for i in range(2):
  for j in range(3):
    if(i == 0 and j == 0):
      train_data, train_label = getData(1, 1)
      continue
    tmp_data, tmp_label = getData(i+1, j+1)
    train_data = np.concatenate([train_data, tmp_data])
    train_label = np.concatenate([train_label, tmp_label])
# user 3 데이터로 검증
for i in range(3):
  if i == 0:
    val_data, val_label = getData(3, i+1)
    continue
  tmp_data, tmp_label = getData(3, i+1)
  val_data = np.concatenate([val_data, tmp_data])
  val_label = np.concatenate([val_label, tmp_label])

print(train_data.shape)
print(train_label.shape)
print(val_data.shape)
print(val_label.shape)

/content/drive/MyDrive/SHLdatasets/user1/day1/Hand_Motion.txt
/content/drive/MyDrive/SHLdatasets/user1/day2/Hand_Motion.txt
/content/drive/MyDrive/SHLdatasets/user1/day3/Hand_Motion.txt
/content/drive/MyDrive/SHLdatasets/user2/day1/Hand_Motion.txt
/content/drive/MyDrive/SHLdatasets/user2/day2/Hand_Motion.txt
/content/drive/MyDrive/SHLdatasets/user2/day3/Hand_Motion.txt
/content/drive/MyDrive/SHLdatasets/user3/day1/Hand_Motion.txt


In [2]:
import numpy as np
train_data = np.load("/content/drive/MyDrive/SHLdatasets/train_data.npy")
val_data = np.load("/content/drive/MyDrive/SHLdatasets/val_data.npy")
train_label = np.load("/content/drive/MyDrive/SHLdatasets/train_label.npy")
val_label = np.load("/content/drive/MyDrive/SHLdatasets/val_label.npy")

In [3]:
mean = np.nanmean(train_data, axis=0)
train_data = train_data - mean
val_data = val_data - mean
std = np.nanstd(train_data, axis=0)
train_data/=std
val_data/=std

In [4]:
train_data[1750000]

array([[-0.62667921, -1.7224187 ,  1.06512092,  0.03283842,  0.11032303,
         0.13222627,  0.01659528,  1.23409338, -0.63570765, -0.33971381,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan]])

In [ ]:
def divide_5sec_10ms(data_10ms, label_10ms):
  data_5sec = np.zeros((0, 500, 17))
  label_5sec = np.zeros((0, 9))
  tmp_data_5sec = np.zeros((0, 17))
  tmp_label_5sec = np.zeros((0, 9))
  cnt = 0
  for i in range(len(data_10ms)):
    if(i%10000==0):
        print(i)
    if cnt == 0:
      label = label_10ms[i]
      reshape_label = label.reshape((1, 9))
      reshape_data_10ms = data_10ms[i][0].reshape((1, 17))
      tmp_data_5sec = np.concatenate([tmp_data_5sec, reshape_data_10ms])
      cnt+=1
      continue
    if not all(label_10ms[i] == label): # 5초 이상 동일 이동수단 유지하지 않으면 버리기.
      tmp_data_5sec = np.zeros((0, 17)) # 0으로 초기화
      tmp_label_5sec = np.zeros((0, 9))
      cnt = 0
      continue
    reshape_data_10ms = data_10ms[i][0].reshape((1, 17))
    tmp_data_5sec = np.concatenate([tmp_data_5sec, reshape_data_10ms])
    if cnt == 499:
      reshape_data = tmp_data_5sec.reshape((1, 500, 17))
      data_5sec = np.concatenate([data_5sec, reshape_data])
      tmp_data_5sec = np.zeros((0, 17)) # 0으로 초기화
      label_5sec = np.concatenate([label_5sec, reshape_label])
      cnt = 0
      continue
    cnt+=1
  return data_5sec, label_5sec

train_data, train_label = divide_5sec_10ms(train_data, train_label)
val_data, val_label = divide_5sec_10ms(val_data, val_label)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

In [ ]:
print(train_data.shape)
print(train_label.shape)
print(val_data.shape)
print(val_label.shape)

In [ ]:
np.save('/content/drive/MyDrive/SHLdatasets/train_data_5sec',train_data)
np.save('/content/drive/MyDrive/SHLdatasets/train_label_5sec',train_label)

np.save('/content/drive/MyDrive/SHLdatasets/val_data_5sec',val_data)
np.save('/content/drive/MyDrive/SHLdatasets/val_label_5sec',val_label)

In [ ]:
train_data = np.nan_to_num(train_data)
val_data = np.nan_to_num(val_data)

In [ ]:
from keras.backend import dropout
from keras.models import Sequential
from keras import layers
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(layers.LSTM(32, input_shape=(500, 12), return_sequences=True))
model.add(layers.LSTM(16, dropout = 0.1, recurrent_dropout = 0.5))
model.add(layers.Dense(9, activation='softmax'))

model.compile(optimizer='rmsprop',
             loss= 'categorical_crossentropy',
             metrics= ['accuracy'])
model.summary()

history = model.fit(train_data,
                     train_label,
                     epochs=50,
                     batch_size=256,
                     validation_data = (val_data, val_label))

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation acc')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()

plt.show()

In [ ]:
import pandas as pd
import numpy as np
from keras.utils.np_utils import to_categorical

def getData(usr, day):
  dir_usr = ["user1", "user2", "user3"]
  dir_day = ["day1", "day2", "day3"]
  file_name = ["Hand_Motion.txt","Hand_GPS.txt","Label.txt"]
  path = "/content/drive/MyDrive/SHLdatasets"

  motion_path = path + "/" + dir_usr[usr-1] + "/" + dir_day[day-1] + "/" + file_name[0]
  gps_path = path + "/" + dir_usr[usr-1] + "/" + dir_day[day-1] + "/" + file_name[1]
  label_path = path + "/" + dir_usr[usr-1] + "/" + dir_day[day-1] + "/" + file_name[2]

  print(motion_path)

  # 'Time', 'Acc_X', 'Acc_Y', 'Acc_Z', 'Gyr_X', 'Gyr_Y', 'Gyr_Z', 'Mag_X', 'Mag_Y', 'Mag_Z'
  full_motion = pd.read_csv(motion_path, sep=" ",header=None, usecols=list(range(10)))
  # 'Time', 'GPS_SNR' , 'GPS_Az, ', 'GPS_Elev', 
  full_gps = pd.read_csv(gps_path, sep=" ",header=None, usecols=[0,4,5,6])
  # Null=0, Still=1, Walking=2, Run=3, Bike=4, Car=5, Bus=6, Train=7, Subway=8
  full_label = pd.read_csv(label_path, sep=" ",header=None, usecols=list(range(2)))

  # NaN 제거
  full_motion = full_motion.fillna(0)
  full_gps = full_gps.fillna(0)

  # 1sec 단위로 자르기
  time = -1
  idx2 = 0
  data_1sec = pd.DataFrame()
  label_1sec = pd.DataFrame()
  tmp1 = pd.DataFrame()
  tmp2 = pd.DataFrame()
  for idx1 in range(len(full_gps)):
    gps_time = full_gps.iloc[idx1, 0]//1000
    if gps_time - time >= 1:
      gap = gps_time - time
      time = gps_time
      idx2 += gap*100 if idx1 != 0 else 0
      if idx2 >= len(full_motion):
        break
      while time != full_motion.iloc[idx2, 0]//1000:
        idx2 += 1
      tmp1 = full_gps.iloc[idx1, 1:].T
      tmp2 = full_motion.iloc[idx2, 1:].T
      data_1sec = data_1sec.append(pd.concat([tmp1, tmp2],ignore_index=True),ignore_index=True)
      label_1sec = label_1sec.append(full_label.iloc[idx2, 1:],ignore_index=True)

  # 넘파이로 바꾸기
  final_data = data_1sec.to_numpy()
  final_data = final_data.reshape(-1,1,12)

  label_np = label_1sec.T
  label_np = label_np.to_numpy()

  # one-hot encoding
  final_label = np.zeros(shape=(len(label_np[0]), 9))
  
  label_one_hot = to_categorical(label_np)
  label_one_hot = label_one_hot[0]
  for i in range(len(label_one_hot)):
    final_label[i] = np.pad(label_one_hot[i], (0,9 - len(label_one_hot[i])), constant_values = 0)

  return final_data, final_label